# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,13.79,87,1,2.24,NZ,1690852342
1,1,adamstown,-25.0660,-130.1015,19.44,60,86,10.32,PN,1690852343
2,2,olonkinbyen,70.9221,-8.7187,6.49,93,99,0.37,SJ,1690852345
3,3,bethel,41.3712,-73.4140,22.64,68,75,0.00,US,1690852346
4,4,arraial do cabo,-22.9661,-42.0278,20.31,81,49,5.70,BR,1690852347


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
)

# Display the map plot
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df['Max Temp'] > 20) & (city_data_df['Max Temp'] < 24)]
ideal_city_data_df = city_data_df.loc[city_data_df['Humidity'] < 40]
ideal_city_data_df = city_data_df.loc[(city_data_df['Wind Speed'] > 5) & (city_data_df['Wind Speed'] < 10)]

ideal_city_data_df = ideal_city_data_df.sort_values(by='Wind Speed', ascending=True)

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
370,370,novyy urgal,51.0736,132.5642,27.29,46,8,5.07,RU,1690852826
139,139,dudinka,69.4058,86.1778,19.88,75,6,5.13,RU,1690852523
217,217,tindouf,27.6711,-8.1474,34.25,12,0,5.14,DZ,1690852623
384,384,lompoc,34.6391,-120.4579,29.08,79,100,5.14,US,1690852644
39,39,nuuk,64.1835,-51.7216,6.34,87,20,5.14,GL,1690852392
...,...,...,...,...,...,...,...,...,...,...
397,397,northam,51.0333,-4.2167,17.03,92,30,9.34,GB,1690852861
199,199,iskushuban,10.2837,50.2300,26.46,58,100,9.41,SO,1690852600
441,441,hobyo,5.3505,48.5268,24.85,82,90,9.56,SO,1690852918
37,37,tual,-5.6667,132.7500,25.98,82,100,9.65,ID,1690852389


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df.copy().reset_index()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,370,370,novyy urgal,51.0736,132.5642,27.29,46,8,5.07,RU,1690852826,
1,139,139,dudinka,69.4058,86.1778,19.88,75,6,5.13,RU,1690852523,
2,217,217,tindouf,27.6711,-8.1474,34.25,12,0,5.14,DZ,1690852623,
3,384,384,lompoc,34.6391,-120.4579,29.08,79,100,5.14,US,1690852644,
4,39,39,nuuk,64.1835,-51.7216,6.34,87,20,5.14,GL,1690852392,
...,...,...,...,...,...,...,...,...,...,...,...,...
118,397,397,northam,51.0333,-4.2167,17.03,92,30,9.34,GB,1690852861,
119,199,199,iskushuban,10.2837,50.2300,26.46,58,100,9.41,SO,1690852600,
120,441,441,hobyo,5.3505,48.5268,24.85,82,90,9.56,SO,1690852918,
121,37,37,tual,-5.6667,132.7500,25.98,82,100,9.65,ID,1690852389,


In [16]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123 entries, 0 to 122
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       123 non-null    int64  
 1   City_ID     123 non-null    int64  
 2   City        123 non-null    object 
 3   Lat         123 non-null    float64
 4   Lng         123 non-null    float64
 5   Max Temp    123 non-null    float64
 6   Humidity    123 non-null    int64  
 7   Cloudiness  123 non-null    int64  
 8   Wind Speed  123 non-null    float64
 9   Country     123 non-null    object 
 10  Date        123 non-null    int64  
 11  Hotel Name  123 non-null    object 
dtypes: float64(4), int64(5), object(3)
memory usage: 11.7+ KB


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
filters= ""
bias = ''
limit = 20
categories = "accommodation.hotel"
params = {
     "categories":categories,
     "filter":filters,
     "bias":bias,
     "limit": limit,
     "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
novyy urgal - nearest hotel: No hotel found
dudinka - nearest hotel: Виктория
tindouf - nearest hotel: محمد بوسبي
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
nuuk - nearest hotel: Nordbo
flying fish cove - nearest hotel: Christmas Island Lodge
ketchikan - nearest hotel: Cape Fox Lodge
nuevo ideal - nearest hotel: No hotel found
lib - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
narsinghgarh - nearest hotel: No hotel found
peterhead - nearest hotel: Albert Hotel
levelland - nearest hotel: Best Inn Texas
invercargill - nearest hotel: The Grand
dunedin - nearest hotel: Scenic Hotel Dunedin City
kribi - nearest hotel: Mont neyanc
coyhaique - nearest hotel: No hotel found
atafu village - nearest hotel: No hotel found
trimbak - nearest hotel: Grape County
puerto colombia - nearest hotel: Hotel Puerto Colombia
baidoa - nearest hotel: No hotel found
kingston - nearest hotel: Aphrodite

,index,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,370,370,novyy urgal,51.0736,132.5642,27.29,46,8,5.07,RU,1690852826,No hotel found
1,139,139,dudinka,69.4058,86.1778,19.88,75,6,5.13,RU,1690852523,Виктория
2,217,217,tindouf,27.6711,-8.1474,34.25,12,0,5.14,DZ,1690852623,محمد بوسبي
3,384,384,lompoc,34.6391,-120.4579,29.08,79,100,5.14,US,1690852644,Embassy Suites by Hilton Lompoc Central Coast
4,39,39,nuuk,64.1835,-51.7216,6.34,87,20,5.14,GL,1690852392,Nordbo
...,...,...,...,...,...,...,...,...,...,...,...,...
118,397,397,northam,51.0333,-4.2167,17.03,92,30,9.34,GB,1690852861,Durrant House Hotel
119,199,199,iskushuban,10.2837,50.2300,26.46,58,100,9.41,SO,1690852600,No hotel found
120,441,441,hobyo,5.3505,48.5268,24.85,82,90,9.56,SO,1690852918,No hotel found
121,37,37,tual,-5.6667,132.7500,25.98,82,100,9.65,ID,1690852389,Suita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE